In [3]:
import mne
import numpy as np
from scipy.signal import welch
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt


# Definição das informações do EEG
n_channels = 8
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=['eeg'] * n_channels)
info.set_montage("standard_1020")

# Carregamento dos dados de glucamon, insulina e teste final
glucamon_files = [
    '../database/s3/glucamon/OpenBCI-RAW-2023-11-08_13-55-20.txt',
    '../database/s3/glucamon/OpenBCI-RAW-2023-11-08_13-57-31.txt'
]
glucamon_data = [np.loadtxt(f, delimiter=',', skiprows=5, usecols=range(1, 9)) for f in glucamon_files]
glucamon_data_combined = np.vstack(glucamon_data)
data_glucamon_cut = glucamon_data_combined[int(0.35 * sfreq):int(200 * sfreq), :]

insulina_files = [
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-19-06.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-19-27.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-20-06.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-21-23.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-28-10.txt'
]
insulina_data = [np.loadtxt(f, delimiter=',', skiprows=5, usecols=range(1, 9)) for f in insulina_files]
insulina_data_combined = np.vstack(insulina_data)

teste_final_file = '../database/s3/teste/OpenBCI-RAW-2023-11-24_12-19-35.txt'
teste_final_data = np.loadtxt(teste_final_file, delimiter=',', skiprows=5, usecols=range(1, 9))
data_basal = teste_final_data[:int(60 * sfreq), :] 

# Criação dos objetos RawArray para MNE
X = {
    'glucamon': mne.io.RawArray(glucamon_data_combined.T, info),
    'insulina': mne.io.RawArray(insulina_data_combined.T, info),
    'teste_basal': mne.io.RawArray(data_basal.T, info),
    'teste': mne.io.RawArray(teste_final_data.T, info),
}

# Filtragem para eliminar ruido e focar nas bandas relevantes
for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

# Definição dos parametros para o processamento em janelas
sr = 250
tamanho_buffer = 5
size = sr * tamanho_buffer
passo = 1 * sr  

# Definição das bandas de frequencia
theta_band = (4, 8)
alpha_band = (8, 13)
beta_band = (13, 30)
gamma_band = (30, 100)

sample = {'focous': [], 'not_focous': []}
results = [0, 0, 0, 0]  # Armazenar contagem de predominancia das bandas

# Processamento dos dados para calcular as bandas e SNR
for key, data in X.items():
    for i in range(0, data.n_times, passo):
        if i + size < data.n_times:
            cut = data.copy().crop(tmin=data.times[i], tmax=data.times[i] + tamanho_buffer)
        else:
            cut = data.copy().crop(tmin=data.times[i], tmax=data.times[-1])

        # Converte o sinal cortado para o dominio da frequencia e calcula a PSD
        cut_data = cut.get_data()
        freqs, psd = welch(cut_data, fs=sr, nperseg=128, noverlap=64)
        psd_avg = np.average(psd, axis=0)

        # Calculo da soma da potência nas bandas theta, alpha, beta e gamma
        theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
        alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
        beta_idxs = np.where((freqs >= beta_band[0]) & (freqs <= beta_band[1]))[0]
        gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

        bands = [
            np.sum(psd_avg[theta_idxs]),
            np.sum(psd_avg[alpha_idxs]),
            np.sum(psd_avg[beta_idxs]),
            np.sum(psd_avg[gamma_idxs])
        ]

        # Identificação da banda dominante
        dominant_band = np.argmax(bands)
        results[dominant_band] += 1

        # Classificação da janela como foco (beta/gamma) ou nao foco
        if dominant_band == 2 or dominant_band == 3:
            sample['focous'].append(psd_avg)
        else:
            sample['not_focous'].append(psd_avg)

    print(f"Resultados para {key}: {results}")

# Normalização dos dados
scaler = StandardScaler()
data_glucamon_normalized = scaler.fit_transform(glucamon_data_combined)
data_insulina_normalized = scaler.fit_transform(insulina_data_combined)
data_teste_basal_normalized = scaler.fit_transform(data_basal)
data_teste_normalized = scaler.fit_transform(teste_final_data)

# Estimativa do ruido de fundo
fft_result_basal = np.fft.fft(data_basal)
psd_basal = np.abs(fft_result_basal) ** 2
base_power = np.mean(psd_basal, axis=1)
estimated_background_noise = np.mean(base_power)

# Calculo do SNR para as janelas cortadas
X_data = []
for psd_avg in sample['focous'] + sample['not_focous']:
    target_amplitudes_adjusted = psd_avg - estimated_background_noise

    # Calculo do narrowband SNR
    narrow_band_SNR = 10 * np.log10(np.abs(target_amplitudes_adjusted) / estimated_background_noise)
    
    # Calculo do wideband SNR
    total_power = np.sum(np.abs(target_amplitudes_adjusted))
    wide_band_SNR = 10 * np.log10(np.abs(target_amplitudes_adjusted) / total_power)

    # Armazenamento das características
    features = [
        np.mean(narrow_band_SNR), np.median(narrow_band_SNR),
        np.mean(wide_band_SNR), np.median(wide_band_SNR)
    ]
    X_data.append(features)

# Definição dos rotulos 
labels = [1] * len(sample['focous']) + [0] * len(sample['not_focous'])

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_data, labels, test_size=0.3, random_state=42)

# Normalização dos dados de treino e teste
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Treinamento do modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)

# Predição e avaliação do modelo
y_pred = svm_model.predict(X_test_scaled)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de classificação:\n", classification_report(y_test, y_pred))

# Implementando o RFE para seleção de características
rfe_selector = RFE(estimator=svm_model, n_features_to_select=10, step=1)
rfe_selector = rfe_selector.fit(X_train_scaled, y_train)

# Visualizando o ranking das características
print("Ranking das características após o RFE:")
print(rfe_selector.ranking_)

# Avaliação após seleção de características
X_train_rfe = rfe_selector.transform(X_train_scaled)
X_test_rfe = rfe_selector.transform(X_test_scaled)

# Reajustar o modelo SVM com as características selecionadas
svm_model_rfe = SVC(kernel='linear')
svm_model_rfe.fit(X_train_rfe, y_train)

# Realizar predições no conjunto de teste com as características selecionadas
y_pred_rfe = svm_model_rfe.predict(X_test_rfe)

# Avaliar o modelo com as características reduzidas
print("Acurácia após RFE:", accuracy_score(y_test, y_pred_rfe))
print("Relatório de classificação após RFE:\n", classification_report(y_test, y_pred_rfe))

#descricao da tarefa 3: eu usei dados de EEG simulados, divididos entre "sinal de foco" e "sinal basal" (que representa o ruido de fundo),
#para calcular a potencia do ruido em cada canal. Para cada janela de 5 segundos, calculei a relacao sinal-ruido (SNR),
#gerando tanto o SNR de banda estreita quanto o de banda larga, com base na diferenca entre o sinal de interesse e o ruido basal.
#Para cada janela, determinei se as bandas beta e gamma eram predominantes. Quando essas bandas dominavam, eu rotulava como "foco",
#e quando nao, como "nao foco". A partir desses dados, extrai a media e a mediana dos SNRs, criando vetores de caracteristicas com 32 colunas
#para cada janela de 5 segundos. Depois disso, dividi os dados em 70% para treinamento e 30% para teste.
#Usei um modelo SVM para classificar se a janela indicava "foco" ou "nao foco". Por fim apliquei o algoritmo RFE para selecionar as caracteristicas mais relevantes e melhorar o desempenho da classificacao.




Creating RawArray with float64 data, n_channels=8, n_times=79959
    Range : 0 ... 79958 =      0.000 ...   319.832 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=366094
    Range : 0 ... 366093 =      0.000 ...  1464.372 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15000
    Range : 0 ... 14999 =      0.000 ...    59.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=137568
    Range : 0 ... 137567 =      0.000 ...   550.268 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50

/home/vinicius/workspace-rec/venv/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 128 is greater than input length  = 94, using nperseg = 94
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Resultados para insulina: [448, 356, 156, 825]
Resultados para teste_basal: [477, 356, 156, 856]


/home/vinicius/workspace-rec/venv/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 128 is greater than input length  = 68, using nperseg = 68
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Resultados para teste: [518, 356, 190, 1332]
Acurácia: 0.6606397774687065
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.56      0.04      0.07       246
           1       0.66      0.99      0.79       473

    accuracy                           0.66       719
   macro avg       0.61      0.51      0.43       719
weighted avg       0.63      0.66      0.54       719



/home/vinicius/workspace-rec/venv/lib/python3.10/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=4. There will be no feature selection and all features will be kept.
  warnings.warn(


Ranking das características após o RFE:
[1 1 1 1]
Acurácia após RFE: 0.6606397774687065
Relatório de classificação após RFE:
               precision    recall  f1-score   support

           0       0.56      0.04      0.07       246
           1       0.66      0.99      0.79       473

    accuracy                           0.66       719
   macro avg       0.61      0.51      0.43       719
weighted avg       0.63      0.66      0.54       719

